# Exploring Airports in Australia

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import geoapify_key

## DataFrame Preparation

In [2]:
# Import cities file as DataFrame
# imported data from a CSV file named "cities.csv" into a Pandas DataFrame called cities_pd.
cities_pd = pd.read_csv("../Resources/cities.csv")
# also displayed the first few rows of the DataFrame using the head() method
cities_pd.head()

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide


In [3]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
# added several new columns to the cities_pd DataFrame to store information
### about airports that you plan to fetch using the Geoapify API
# initialized these columns with empty strings
cities_pd["Lat"] = ""
cities_pd["Lon"] = ""
cities_pd["Airport Name"] = ""
cities_pd["IATA Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Distance"] = ""
cities_pd["Website"] = ""
cities_pd.head()

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,


## Look for Geographical Coordinates for Each City

In [4]:
# Define the API parameters
# You've created a params dictionary that includes your
### API key and specifies the response format as JSON
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
# You've set the base URL for the Geoapify API to
###  "https://api.geoapify.com/v1/geocode/search".
base_url = "https://api.geoapify.com/v1/geocode/search"

In [5]:
# Print a message to follow up the airport search
#! code is preparing the DataFrame with latitude and longitude information, which you can later
###! use to fetch airport data from Geoapify

#! print a message to indicate that you are starting the airport search.
print("Starting airport search")

# Loop through the cities_pd DataFrame and search coordinates for each city

#! loop through the cities_pd DataFrame, which contains city names, and search for the coordinates 
###! (latitude and longitude) for each city
for index, row in cities_pd.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    #!!! creating a new string variable city by concatenating the value in the "City" column of the
    #!!! current row in the cities_pd DataFrame with the string ", Australia".
    city = row["City"] + ", Australia"

    # Add the current city to the parameters
    #!!! updating the "text" parameter in the params dictionary with the value of the city variable
    params["text"] = f"{city}"

    # Make the API request
    #!!! you are making an HTTP GET request to the specified base_url with parameters defined in the
    #!!! params dictionary
    #@@@ <base_url>: This is the base URL for the API endpoint you want to access. In your code, it's 
    #@@@ set to the Geoapify API endpoint for geocoding search.
    #$$$ <requests.get()>: This function is from the requests library and is used to send an HTTP GET 
    #$$$ request to the specified URL.
    #%%% response variable. This response typically includes information such as the status code, headers, 
    #%%% and the content of the response
    response = requests.get(base_url, params=params)

    # Convert response to JSON
    #!!! used to parse the content of the HTTP response obtained from the API request and convert it from a
    #!!! JSON-formatted string into a Python dictionary
    response = response.json()

    # Extract latitude and longitude
    #!!! updating the value in the "Lat" column of the cities_pd DataFrame for the current index with the 
    #!!! latitude value obtained from the API response
    cities_pd.loc[index, "Lat"] = response["results"][0]["lat"]
    #!!! updating the value in the "Lon" column of the cities_pd DataFrame for the current index with the 
    #!!!longitude value obtained from the API response.
    cities_pd.loc[index, "Lon"] = response["results"][0]["lon"]
    
    # Log the search results
    #!!! debugging or logging statement used to display a message indicating that the coordinates for a
    #!!! specific city have been successfully fetched
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
cities_pd.head()

Starting airport search


Coordinates for Sydney, Australia fetched...
Coordinates for Melbourne, Australia fetched...
Coordinates for Perth, Australia fetched...
Coordinates for Cairns, Australia fetched...
Coordinates for Adelaide, Australia fetched...
Coordinates for Gold Coast, Australia fetched...
Coordinates for Townsville, Australia fetched...
Coordinates for Williamtown, Australia fetched...


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,,,,,
1,Melbourne,-37.814245,144.963173,,,,,
2,Perth,-31.955893,115.860585,,,,,
3,Cairns,-16.920666,145.772185,,,,,
4,Adelaide,-34.928181,138.599931,,,,,


## Retrieve Airports' Information

In [6]:
####### Set parameters to search for airport's details

#! set a radius of 50,000 meters
radius = 50000
#! dictionary containing additional parameters
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

######## Print a message to follow up the airport search
print("Starting airport details search")

######## Iterate through the types_df DataFrame

#! for loop that iterates through the rows of the cities_pd Pandas DataFrame
for index, row in cities_pd.iterrows():
    
    ######## Get the city's name
    #! extracting the value in the ["City"] column of the current 'row' (the row being iterated over)
    #! and assigning it to the variable 'city'.
    city = row["City"]
    
    ######## Get latitude, longitude from the DataFrame
    #! extracting the value in the ["Lat"] column of the current 'row' (the row being iterated over)
    #! and assigning it to the variable 'latitude'.
    latitude = row["Lat"]
    #! extracting the value in the ["Lon"] column of the current 'row' (the row being iterated over)
    #! and assigning it to the variable 'longitude'.
    longitude = row["Lon"]
    
    ######## Add the current city's latitude and longitude to the params dictionary
    #! updating the ["filter"] parameter in the 'params' dictionary with a filter specification (<circle:>) based on
    #! the current city's coordinates ({longitude} and {latitude}) and the specified {radius}
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    #! updating the ["bias"] parameter in the 'params' dictionary with a filter specification (<proximity:>) based on
    #! the current city's coordinates ({longitude} and {latitude})
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    ######## Set base URL
    #! setting the base_url variable to the URL of the Geoapify API endpoint that you want to make requests to
    base_url = "https://api.geoapify.com/v2/places"
           
    ######## Make an API request using the params dictionary
    #! making an HTTP GET request to the Geoapify API using the specified base_url and parameters provided in the params dictionary
    response = requests.get(base_url, params=params)
        
    ######## Convert the API response to JSON format
    # # <.json()> used to parse the JSON content from the HTTP response
    ### and store request as a Python data structure
    response = response.json()
    
    ######## Grab the first airport from the results and store the details in the DataFrame
    try:
        cities_pd.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        cities_pd.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        cities_pd.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        cities_pd.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        cities_pd.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        ######## If no airport is found, log the error.
        #! printing an error message in case an exception (e) is raised during your code execution
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")
        
    ######## Log the search results
    #! print a message indicating the nearest airport to a specific city based on the data retrieved from the Geoapify API
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")

######## Display sample data
cities_pd

Starting airport details search
nearest airport from Sydney: Sydney (Kingsford Smith) Airport
nearest airport from Melbourne: Essendon Fields Airport
nearest airport from Perth: Perth Airport
website not found for Cairns Airport
nearest airport from Cairns: Cairns Airport
nearest airport from Adelaide: Adelaide Airport
nearest airport from Gold Coast: Gold Coast Airport
nearest airport from Townsville: Townsville Airport
nearest airport from Williamtown: Newcastle Airport


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,Sydney (Kingsford Smith) Airport,SYD,"Foreshore Road Offramp, Botany NSW 2019, Austr...",6742,https://www.sydneyairport.com.au/
1,Melbourne,-37.814245,144.963173,Essendon Fields Airport,MEB,"72 Hargrave Avenue, Essendon Fields VIC 3041, ...",9983,https://www.ef.com.au/
2,Perth,-31.955893,115.860585,Perth Airport,PER,"Perth Airport WA 6105, Australia",8775,https://www.perthairport.com.au/
3,Cairns,-16.920666,145.772185,Cairns Airport,CNS,"Airport Avenue, Aeroglen QLD 4870, Australia",3461,
4,Adelaide,-34.928181,138.599931,Adelaide Airport,ADL,"Adelaide Airport SA 5950, Australia",4898,https://www.adelaideairport.com.au/
5,Gold Coast,-28.002373,153.414599,Gold Coast Airport,OOL,"Loongana Avenue, Bilinga QLD 4225, Australia",18807,https://www.goldcoastairport.com.au/
6,Townsville,-19.256939,146.823954,Townsville Airport,TSV,"Old Common Road, Rowes Bay QLD 4810, Australia",4711,https://www.townsvilleairport.com.au/
7,Williamtown,-32.815,151.842778,Newcastle Airport,NTL,"55 Slades Road, Williamtown NSW 2318, Australia",620,https://www.newcastleairport.com.au


In [7]:
# Save Data to CSV
#! used to export the contents of the cities_pd Pandas DataFrame to 
#! a CSV (Comma-Separated Values) file named "Airport_Output.csv."
cities_pd.to_csv("Airport_Output.csv", index=False)